# Prony series identification for linear viscoelastic material models
M. Springer | 2022 v0.1 

***


## Background - Linear viscoelasticity

Theory of linear viscoelasticity is used to describe the mechanical response of polymer materials under small strain conditions. An extensive review can be found in [Brinson H. F., Brinson L. C,. Polymer Engineering Science and Viscoelasticity,
Springer, 2015]. The summary below was extracted from [Springer, M., and Bosco N. Progress in Photovoltaics: Research and Applications 28.7 (2020): 659-681]

The rate-dependent material behavior can be described in either the time or frequency domain. In the time domain, the uniaxial, nonaging, isothermal stress-strain equation for a linear viscoelastic material can be represented by a Boltzmann superposition integral,

\begin{equation}
\sigma(t) = \int_0^t E(t-\tau) \frac{\mathrm{d \varepsilon(\tau)}}{\mathrm{d} \tau} \mathrm{d} \tau \quad ,
\end{equation}

where $\sigma(t)$ is the stress response over time, $t$; $E(t)$ is the relaxation modulus; $\varepsilon$ denotes the strain; and $\tau$ is the integration variable. The limiting moduli for the viscoelastic material are defined as the instantaneous modulus, $E(t=0) = E_0$, and the equilibrium modulus, i.e., $E(t) \rightarrow E_{\infty}$ for $t \rightarrow \infty$. 

The Generalized Maxwell Model is commonly used to represent the stress-strain response of polymers. The relaxation modulus derived from this model is given by,

\begin{equation}
E(t) = E_{\infty} + \sum\limits_{i=1}^{m} E_i \exp\left(-\frac{t}{\tau_i} \right) \quad ,
\label{eq:RelProny}
\end{equation}

where $\tau_i$ (relaxation times),  $E_i = E_0 \alpha_i$ (relaxation moduli) are material properties, and $m$ is the number of terms in the series. The above equation is often referred to as Prony series, and the equilibrium modulus can be defined by the Prony series as,

\begin{equation}
	E_{\infty} = E_0 \left[1-\sum_{i=1}^N \alpha_i \right] \quad .
\end{equation}

The material properties can be directly obtained from relaxation or frequency-dependent test data. Material properties measured in the time domain can be converted into the frequency domain, and vice versa, by making use of a Fourier transformation,

\begin{align}
E'(\omega) &= E_{\infty} + \sum\limits_{i=1}^{m}\frac{\omega^2 \tau_i^2 E_i}{\omega^2 \tau_i^2 + 1} \quad ,
\label{eq:PronyStor} \\
E''(\omega) &= \sum\limits_{i=1}^{m} \frac{\omega \tau_i E_i}{\omega^2 \tau_i^2 +1} \quad .
\label{eq:PronyLoss}
\end{align}

Herein, $E'(\omega)$ is the storage modulus, $E''(\omega)$ is the loss modulus, and $\omega = 2 \pi f$ is the angular frequency, where $f=1/t$ is the frequency in Hertz and $t$ is the time period in seconds, respectively. The complex modulus, $E^*(\omega)$, and the loss factor, $\tan(\delta)$, are given as,

\begin{align}
E^*(\omega) &= E'(\omega)  + i E''(\omega) \quad \text{and} \\
\tan(\delta)  & =  \frac{E''(\omega)}{E'(\omega)} \quad ,
\end{align}

respectively.

For thermo-rheologically simple materials the time-temperature superposition principle (TTSP) can be applied. For such materials, the viscoelastic response at one temperature is related to the viscoelastic response at another temperature
by changing the time scale (or frequency). This way, the time scale for the materials characterization can be extended by conducting the same frequency measurements at different temperatures. Afterward, a reference temperature is selected, and the isothermal measurements are shifted on a logarithmic time (or frequency) scale to form a so–called master curve.

![TTSP](figures/TTSP.png){:height="50%" width="50%"}

<img src="figures/TTSP.png" width="500" >

In [15]:
%matplotlib widget

import LinViscoFit as visco
#import ipywidgets as widgets

#import pandas as pd
#import io
#import os

#from IPython.display import display, clear_output, HTML
#from base64 import b64encode
#import matplotlib.pyplot as plt
#pd.set_option('display.float_format', '{:.5e}'.format)

visco.format_fig()
GUI = visco.GUIControl()

***
## Parameter estimation

The linear viscoelastic curve fitting routine in this notebook allos for experimental data in both the time and frequency domains. The following code convetions are used within this notebook:

| Physical quantity      | Symbol          | Variable       |
| :--------------------- | :-------------: | :------------- |
| Relaxation modulus:    | $E(t)$          | ```E_relax```  |
| Storage modulus:       | $E'(\omega)$    | ```E_stor```   |
| Loss modulus:          | $E''(\omega)$   | ```E_loss```   |
| Complex modulus:       | $|E^{*}|$       | ```E_comp```   |
| Loss modulus:          | $\tan(\delta)$  | ```tan_del```  |
| Instantaneous modulus: | $E_0$           | ```E_0```      |
| Equilibrium modulus:   | $E_{inf}$       | ```E_inf```    |
| Angular frequency:     | $\omega$        | ```omega```    |
| Frequency:             | $f$             | ```f```        |
| Time:                  | $t$             | ```t```        |
| Temperature:           | $\theta$        | ```T```        |


### Specify and upload input data

#### Domain
This notebook allows for the estimation of Prony series parameters used in a Generalized Maxwell model to describe the linear viscoelastic behavior of polymer materials. The parameters can be either fitted from measurement data of Dynamic mechanical thermal analysis (DMTA) in the frequency domain (freq) or from relaxation experiments in the time domain (time). 

#### Instrument
DMTA measurements conducted with a Netzsch Gabo DMA EPLEXOR (Eplexor) can be directly uploaded in this notebook as Excel files. For measurements conducted with other instruments choose the user option (user) and preprocess the input files to be provided in comma-separated values (csv) format with the following columns header:
* **Frequency domain:** header = ```f, E_stor, E_loss```, where ```f``` is the frequency in Hertz (Hz), and ```E_stor``` and *E_loss* are the storage and loss modulus in Megapascal (MPa), respectively.
* **Time domain:** header = ```t, E_relax```, where ```t``` is the time in seconds (s), and ```E_relax``` is the relaxation modulus in MPa.

#### Type
The data can be provided as measurement sets at different temperatures (raw) or as master curve obtained from time-temperature superposition (master). If measurements at different temperatures are provided (raw) the individual temperature sets need to be identified. 
* **Eplexor** The notebook identifies the corresponding temperature sets automatically (only available in the frequency domain).
* **user** An additional column with the header 'Set' is necessary to identify the measurement sets from the input file. All measurements of the same temperature set are marked with the same number, e.g. 1 for all measurements in the the first temperature set. The first tempeature set (1) represents the coldest temperature followed by the second set (2) at the next higher temperature and so forth (see the provided example input files for further details).

[Example input files](https://github.com/martin-springer/LinViscoFit/tree/main/examples)

In [2]:
display(GUI.w_inp_gen)

#### Reference temperature 

Reference temperature refers to the temperature for which the master curve has been created through time-temperature superpostion. 

If a master curve from the Eplexor software is provided the reference temperature will be automatically extracted. 

For all other options please provide either the reference temperature of the user-specified master curve or the desired reference temperature for the time-temperature superposition of the raw data (for raw data the desired reference temperature will be automatically adjusted to align with the closest provided temperature set of the measurement input file).

#### Optional shift factor upload
If master curves are provided, the shift factor upload is optional and allows for the calculation of polynomial (D1 to D4) shift functions and the Williams–Landel–Ferry (WLF) shift function, but is not required for the Prony series estimation. 

If a master curve from created by the Eplexor software is provided, the default behavior of the notebook is to use the WLF shift function from the Eplexor software. However, in the time-temperature superpostion section, a checkbox is provided to overwrite the WLF fit of the Eplexor software and conduct another WLF fit with the algorithm in this notebook, which is then used to estimate the Prony series parameters.

If raw measurement data are provided (for both Eplexor or user), the shift factors can be either determined for the desired reference temperature in the time-temperature superposition section or user-specified shift factors can be uploaded to be used to create the master curve. 

In [3]:
display(GUI.w_inp_shift)

#### Load and check the provided input files

Once all the input parameter are specified and the necessary files are uploaded click the *Load data* button below. If the input files could be loaded successfully, the widgets will indicate the state of the uploaded data.

In [4]:
display(GUI.w_inp_load)
display(GUI.w_inp_check)

***
### Time-temperature superposition (shift functions)

This section allows to either plot the provided shift factors and shift functions for master curve inputs or allows for determining shift factors and functions from raw input data.

#### Shift factors log(a_T)

In [5]:
display(GUI.w_aT)

#### Shift functions (WLF & Polynomial degree 1 to 4)

In [6]:
display(GUI.w_shift)

***
### Pre-process (smooth) master curve

A moving median filter to remove outliers in the measurements can be applied before the Prony series terms are estimated. The window size (win) can be adjusted through the slider. A window size of 1 means that no filtering procedure is performed and the raw data are fitted. 

In [7]:
display(GUI.b_smooth)
display(GUI.out_smooth)

Button(button_style='info', description='smooth master curve', layout=Layout(height='auto', width='200px'), st…

Output()

***
### Estimate Prony series parameters

#### Define the number and discretization of the Prony terms

The default behavior is to apply an equidistant logarithmical spacing along the time axis and use one Prony term per decade. 

In [8]:
# apply filter
display(GUI.b_dis)
display(GUI.out_dis)

Button(button_style='info', description='plot discretization', layout=Layout(height='auto', width='200px'), st…

Output()

#### Curve fitting

Frequency domain: GUSTL
Time domain: least squares optimization

In [9]:
# fit Prony series
display(GUI.b_fit)
display(GUI.out_fit)

Button(button_style='danger', description='fit Prony series', layout=Layout(height='auto', width='200px'), sty…

Output()

***
### Generalized Maxwell model

In [10]:
display(GUI.b_GMaxw)
display(GUI.out_GMaxw)


Button(button_style='info', description='Plot Gen. Maxwell', layout=Layout(height='auto', width='200px'), styl…

Output()

***
### Download results

In [11]:
display(GUI.db_zip)

Button(button_style='warning', description='Download zip', layout=Layout(height='auto', width='200px'), style=…

In [12]:
#old
display(GUI.db_prony)

Button(button_style='warning', description='Download Prony series', layout=Layout(height='auto', width='200px'…

***
### Start over

In [13]:
display(GUI.b_reload)
display(GUI.out_html)

Button(button_style='danger', description='Clear notebook!', layout=Layout(height='auto', width='200px'), styl…

Output()

***
## Verification - Compare with Ansys